In [14]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery
from googleapiclient import errors
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
import json
import os

from tqdm.auto import tqdm
tqdm.pandas(leave=False)

# Training

## configuration

In [8]:
PROJECT                  = "mlteam-ml-specialization-2021"
TRAIN_PATH               = "gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/train.csv"
EVAL_PATH                = "gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/test/evalset.csv"
TEST_PATH                = "gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/test/testset.csv"
MODEL_NAME               = "011_hptuning_lr_epochs"
BATCH_SIZE               = 30000 #1 epoch -> 3 steps
NUM_EPOCHS               = 500
SCANN_NUM_NEIGHBORS      = 100
#LEARNING_RATE            = 0.01
EMBEDDING_DIM            = 128
USER_FEATURES            = ["Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years", "Marital_Status"]
USER_INPUT_EMBEDDING_DIM = 8
FORCE                    = True

In [9]:
%%writefile config.yaml
trainingInput:
  hyperparameters:
    goal: MINIMIZE
    hyperparameterMetricTag: epoch_total_loss
    maxTrials: 9
    maxParallelTrials: 9
    enableTrialEarlyStopping: False
    params:
    - parameterName: num-epochs
      type: DISCRETE
      discreteValues:
      - 10
      - 50
      - 200
    - parameterName: learning-rate
      type: DISCRETE
      discreteValues:
      - 0.0001
      - 0.001
      - 0.01
    - parameterName: product-input-embedding-dim
      type: DISCRETE
      discreteValues:
      - 128
    - parameterName: embedding-dim
      type: DISCRETE
      discreteValues:
      - 128
    - parameterName: common-layers
      type: CATEGORICAL
      categoricalValues:
      - '[256]'
    - parameterName: product-features
      type: CATEGORICAL
      categoricalValues:
      - '["Product_ID", "Product_Category_1"]'

Writing config.yaml


In [10]:
MODEL_PATH = os.path.join("gs://mlteam-ml-specialization-2021-blackfriday/aiplatform_jobs", MODEL_NAME)
if tf.io.gfile.exists(MODEL_PATH):
    if FORCE:
        tf.io.gfile.rmtree(MODEL_PATH)
    else:
        assert not tf.io.gfile.exists(MODEL_PATH), f"{MODEL_PATH} already exists, set FORCE=True to overwrite"


os.environ["TRAIN_PATH"]               = str(TRAIN_PATH)
os.environ["EVAL_PATH"]                = str(EVAL_PATH)
os.environ["TEST_PATH"]                = str(TEST_PATH)
os.environ["MODEL_NAME"]               = str(MODEL_NAME)
os.environ["MODEL_PATH"]               = str(MODEL_PATH)
os.environ["BATCH_SIZE"]               = str(BATCH_SIZE)
#os.environ["NUM_EPOCHS"]               = str(NUM_EPOCHS)
os.environ["SCANN_NUM_NEIGHBORS"]      = str(SCANN_NUM_NEIGHBORS)
#os.environ["LEARNING_RATE"]            = str(LEARNING_RATE)
os.environ["EMBEDDING_DIM"]            = str(EMBEDDING_DIM)
os.environ["USER_INPUT_EMBEDDING_DIM"] = str(USER_INPUT_EMBEDDING_DIM)
os.environ["USER_FEATURES"]            = json.dumps(USER_FEATURES)

## launch

In [11]:
%%bash

# JOB_NAME: the name of your job running on AI Platform.
JOB_NAME=bf_${MODEL_NAME}_$(date +%Y%m%d_%H%M%S)

# REGION: select a region from https://cloud.google.com/ai-platform/training/docs/regions
# or use the default '`us-central1`'. The region is where the model will be deployed.
REGION=europe-west1
PYTHON_VERSION=3.7
RUNTIME_VERSION=2.4

current_dir="$(cd "$(dirname "${BASH_SOURCE[0]}")" >/dev/null 2>&1 && pwd)"
cd ${current_dir}"/../../.."

config_file="${current_dir}/config.yaml"

gcloud ai-platform jobs submit training "${JOB_NAME}" \
  --package-path trainer/ \
  --module-name trainer.task \
  --region ${REGION} \
  --python-version ${PYTHON_VERSION} \
  --runtime-version ${RUNTIME_VERSION} \
  --job-dir "${MODEL_PATH}" \
  --config "${config_file}" \
  -- \
  --train-path="${TRAIN_PATH}" \
  --eval-path="${EVAL_PATH}" \
  --job-dir="${MODEL_PATH}" \
  --batch-size=${BATCH_SIZE} \
  --scann-num-neighbors=${SCANN_NUM_NEIGHBORS} \
  --user-features="${USER_FEATURES}" \
  --embedding-dim=${EMBEDDING_DIM} \
  --user-input-embedding-dim=${USER_INPUT_EMBEDDING_DIM}

#  --num-epochs=${NUM_EPOCHS} \
#  --learning-rate=${LEARNING_RATE} \

gcloud ai-platform jobs describe ${JOB_NAME}

jobId: bf_011_hptuning_lr_epochs_20210604_100012
state: QUEUED
createTime: '2021-06-04T10:00:15Z'
etag: oQQCKkiRBk4=
jobId: bf_011_hptuning_lr_epochs_20210604_100012
state: PREPARING
trainingInput:
  args:
  - --train-path=gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/train.csv
  - --eval-path=gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/test/evalset.csv
  - --job-dir=gs://mlteam-ml-specialization-2021-blackfriday/aiplatform_jobs/011_hptuning_lr_epochs
  - --batch-size=30000
  - --scann-num-neighbors=100
  - --user-features=["Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years",
    "Marital_Status"]
  - --embedding-dim=128
  - --user-input-embedding-dim=8
  hyperparameters:
    goal: MINIMIZE
    hyperparameterMetricTag: epoch_total_loss
    maxParallelTrials: 9
    maxTrials: 9
    params:
    - discreteValues:
      - 10.0
      - 50.0
      - 200.0
      parameterName: num-epochs
      type: DISCRETE


Job [bf_011_hptuning_lr_epochs_20210604_100012] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe bf_011_hptuning_lr_epochs_20210604_100012

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs bf_011_hptuning_lr_epochs_20210604_100012

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/bf_011_hptuning_lr_epochs_20210604_100012?project=mlteam-ml-specialization-2021

View logs at:
https://console.cloud.google.com/logs?resource=ml_job%2Fjob_id%2Fbf_011_hptuning_lr_epochs_20210604_100012&project=mlteam-ml-specialization-2021


In [12]:
!rm -rf config.yaml

In [15]:
metadata_path = os.path.join("gs://mlteam-ml-specialization-2021-blackfriday/aiplatform_jobs", MODEL_NAME, "metadata.json")

ml               = discovery.build('ml','v1')
projectId        = 'projects/{}'.format(PROJECT)
ai_platform_data = ml.projects().jobs().list(parent=projectId).execute()
jobs             = ai_platform_data["jobs"]
latest_job       = sorted([j for j in jobs if j['jobId'].startswith(f"bf_{MODEL_NAME}")], key=lambda x: x["jobId"])[-1]
if latest_job["trainingOutput"].get('isHyperparameterTuningJob',None) is not None:
    trials = latest_job["trainingOutput"]["trials"]
    json.dump(trials, tf.io.gfile.GFile(metadata_path, "w"))
else:
    metadata = [{
        'trialId'        : '1',
        'hyperparameters': {},
        'startTime'      : latest_job["startTime"],
        'endTime'        : latest_job["endTime"],
        'state'          : latest_job["state"],
    }]
    json.dump(metadata, tf.io.gfile.GFile(metadata_path, "w"))

# Evaluation

## ground truth

In [16]:
ALL_USER_FEATURES   = ["Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years", "Marital_Status"] 

df = pd.read_csv(TEST_PATH)

## baseline

In [17]:
class BlackFridayBaseline:
    def __init__(self, trainset_location, user_features_list, build_inplace=False):
        self.trainset_location, self.user_features_list = trainset_location, user_features_list
        if build_inplace:
            self.build()
        
    def build(self):
        self.df = pd.read_csv(self.trainset_location).groupby(
            self.user_features_list+["Product_ID"])["User_ID"].count().reset_index().rename(
            columns={"User_ID":"count_product"})
        
    def predict(self, user_features, n_products):
        mask = None
        for k,v in user_features.items():
            if mask is None:
                mask = self.df[k]==v
            else:
                mask = mask&(self.df[k]==v)
        return self.df[mask].sort_values(self.user_features_list+["count_product"])["Product_ID"].values[:n_products]
        

## model(s)

In [18]:
class PredictionModel:
    def __init__(self, model_path, build_inplace=False):
        self.model_path=model_path
        if build_inplace:
            self.build()
    
    def build(self):
        self.model=tf.keras.models.load_model(self.model_path)
        
    def predict_batch(self, model_input, n_products):
        scores, products = self.model({k:tf.constant(v) for k,v in model_input.items()})
        return products.numpy()[:,:n_products]
    
    def predict_single(self, model_input, n_products):
        scores, products = self.model({k:tf.constant([v]) for k,v in model_input.items()})
        return products.numpy()[0,:n_products]
    


    

## stats

In [19]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
df_grouped = pd.read_csv(TEST_PATH).groupby(ALL_USER_FEATURES)["Product_ID"].apply(list).apply(np.array).reset_index()
top_k = [1, 10, 50, 100]
results = dict()

baseline = BlackFridayBaseline(TRAIN_PATH, ALL_USER_FEATURES, True)
for k in tqdm(top_k, leave=False):
    metrics_id = f"baseline_top{k}"
    df_grouped[metrics_id]         = df_grouped[ALL_USER_FEATURES].progress_apply(lambda x: baseline.predict(x, k), axis=1)
    baseline_true_positives        = df_grouped.progress_apply(lambda x: np.intersect1d(x["Product_ID"],x[metrics_id]).shape[0], axis=1).sum()
    baseline_false_negatives       = df.shape[0] - baseline_true_positives
    baseline_false_positives       = df_grouped[metrics_id].apply(lambda x: x.shape[0]).sum() - baseline_true_positives
    baseline_reach                 = df_grouped.progress_apply(lambda x: min(x["Product_ID"].shape[0],x[metrics_id].shape[0]), axis=1).sum()
    results[f"baseline_top{k}"]    = {
        "top"  : k,
        "model": "baseline",
        "tp"   : baseline_true_positives,
        "fp"   : baseline_false_positives,
        "fn"   : baseline_false_negatives,
        "reach": baseline_reach #portata
    }
    

metadata_path = os.path.join("gs://mlteam-ml-specialization-2021-blackfriday/aiplatform_jobs", MODEL_NAME, "metadata.json")
metadata = json.load(tf.io.gfile.GFile(metadata_path, "r"))
for j in tqdm(metadata, leave=False):
    model = PredictionModel(os.path.join("gs://mlteam-ml-specialization-2021-blackfriday/aiplatform_jobs", MODEL_NAME, j['trialId'], 'Scann'), True)
    hps = json.dumps(j["hyperparameters"])
    model_id = f"trial_{j['trialId']}"
    for k in tqdm(top_k, leave=False):
        metrics_id=f"trial_{j['trialId']}_top{k}"
        df_grouped[metrics_id] = df_grouped[USER_FEATURES].progress_apply(lambda x: model.predict_single(x.astype(str), k).astype(str), axis=1)
        tp = df_grouped.progress_apply(lambda x: np.intersect1d(x["Product_ID"],x[metrics_id]).shape[0], axis=1).sum()
        fn = df.shape[0] - tp
        fp = df_grouped[metrics_id].apply(lambda x: x.shape[0]).sum() - tp
        reach = df_grouped.progress_apply(lambda x: min(len(x["Product_ID"]),x[metrics_id].shape[0]), axis=1).sum() 
        results[metrics_id] = {
            "top"  : k,
            "model": model_id,
            "tp"   : tp,
            "fp"   : fp,
            "fn"   : fn,
            "reach": reach
        }
    
df_results = pd.DataFrame(results).T
df_results["precision"] = df_results["tp"]/(df_results["tp"]+df_results["fp"])
df_results["recall"] = df_results["tp"]/(df_results["tp"]+df_results["fn"])
df_results["tp_over_reach"] = df_results["tp"]/df_results["reach"]
df_results

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

...

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

,top,model,tp,fp,fn,reach,precision,recall,tp_over_reach
baseline_top1,1,baseline,637,1754,273647,2391,0.266416,0.002322,0.266416
baseline_top10,10,baseline,5184,14437,269100,19618,0.264207,0.0189,0.264247
baseline_top50,50,baseline,15198,39057,259086,54252,0.280122,0.05541,0.280137
baseline_top100,100,baseline,19780,47500,254504,67277,0.293995,0.072115,0.294008
trial_9_top1,1,trial_9,92,2338,274192,2430,0.03786,0.000335,0.03786
trial_9_top10,10,trial_9,1078,23222,273206,23612,0.044362,0.00393,0.045655
trial_9_top50,50,trial_9,6914,114586,267370,91899,0.056905,0.025207,0.075235
trial_9_top100,100,trial_9,15173,227827,259111,146329,0.06244,0.055319,0.103691
trial_8_top1,1,trial_8,102,2328,274182,2430,0.041975,0.000372,0.041975
trial_8_top10,10,trial_8,877,23423,273407,23612,0.036091,0.003197,0.037142


In [20]:
df_results.pivot("model","top","precision")

top,1,10,50,100
model,,,,
baseline,0.266416,0.264207,0.280122,0.293995
trial_1,0.111934,0.084403,0.063556,0.057901
trial_2,0.041564,0.034198,0.033226,0.032293
trial_3,0.046502,0.048272,0.045704,0.044663
trial_4,0.027984,0.017449,0.026247,0.03384
trial_5,0.017284,0.02214,0.025909,0.027786
trial_6,0.011934,0.008066,0.00916,0.013835
trial_7,0.06214,0.064444,0.05623,0.052728
trial_8,0.041975,0.036091,0.033531,0.032848


In [21]:
df_results.pivot("model","top","recall")

top,1,10,50,100
model,,,,
baseline,0.002322,0.0189,0.05541,0.072115
trial_1,0.000992,0.007478,0.028153,0.051297
trial_2,0.000368,0.00303,0.014718,0.028605
trial_3,0.000412,0.004277,0.020245,0.039568
trial_4,0.000248,0.001546,0.011627,0.02998
trial_5,0.000153,0.001961,0.011477,0.024617
trial_6,0.000106,0.000715,0.004058,0.012257
trial_7,0.000551,0.005709,0.024908,0.046714
trial_8,0.000372,0.003197,0.014853,0.029101


In [22]:
df_results.pivot("model","top","tp_over_reach")

top,1,10,50,100
model,,,,
baseline,0.266416,0.264247,0.280137,0.294008
trial_1,0.111934,0.086863,0.084027,0.096153
trial_2,0.041564,0.035194,0.043929,0.053631
trial_3,0.046502,0.049678,0.060425,0.074168
trial_4,0.027984,0.017957,0.034701,0.056195
trial_5,0.017284,0.022785,0.034255,0.046143
trial_6,0.011934,0.008301,0.012111,0.022976
trial_7,0.06214,0.066322,0.074342,0.087563
trial_8,0.041975,0.037142,0.044331,0.054548
